<a href="https://colab.research.google.com/github/phillipSloan/ComputationalLogic/blob/prolexa-plus/Report%20Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install SWI-Prolog

Installing non-python dependeincies can be achieved via '!apt-get'

In [1]:
!apt-get install swi-prolog -qqq > /dev/null

Extracting templates from packages: 100%


### Install Prolexa

Install the Prolexa-plus branch directly from Github via pip. Note that once you have forked the Github repo and made changes you will have to change this link to reference your own repo. 

In [2]:
!yes | pip install git+https://github.com/phillipSloan/ComputationalLogic/ -qqq > /dev/null

  ERROR: Failed building wheel for prolexa
  DEPRECATION: prolexa was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.3.6 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 3.10.1 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


### Instantiate Prolexa 


In [3]:
from pyswip import Prolog
import prolexa.meta_grammar as meta

# Added this due to an error with meta
import nltk
nltk.download('omw-1.4')
  

pl = Prolog()
meta.reset_grammar()
meta.initialise_prolexa(pl)

2021-12-27 22:36:23,664 --------------------------------------------------------------------------------
2021-12-27 22:36:23,667 The model key 'pos' now maps to 'https://huggingface.co/flair/pos-english' on the HuggingFace ModelHub
2021-12-27 22:36:23,670  - The most current version of the model is automatically downloaded from there.
2021-12-27 22:36:23,673  - (you can alternatively manually download the original model at https://nlp.informatik.hu-berlin.de/resources/models/pos/en-pos-ontonotes-v0.5.pt)
2021-12-27 22:36:23,675 --------------------------------------------------------------------------------
2021-12-27 22:36:24,015 loading file /root/.flair/models/pos-english/a9a73f6cd878edce8a0fa518db76f441f1cc49c2525b2b4557af278ec2f0659e.121306ea62993d04cd1978398b68396931a39eb47754c8a06a87f325ea70ac63


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


### First we add a few rules
First the traditional one that works:

In [4]:
_ = meta.standardised_query(pl, "peter is human")[0]['Output']
_ = meta.standardised_query(pl, "every human is mortal")[0]['Output']

print(meta.standardised_query(pl, "explain why peter is mortal")[0]['Output'])

peter is human; every human is mortal; therefore peter is mortal


Now using the new prove_rb meta-interpreter:

In [5]:
_ = meta.standardised_query(pl, "donald is happy")[0]['Output']
_ = meta.standardised_query(pl, "every teacher is happy")[0]['Output']

print(meta.standardised_query(pl, "explain why donald is a teacher")[0]['Output'])

donald is happy; every teacher is happy; therefore donald is a teacher


Demonstrating that the meta-interpreter doesn't fall into an infinite recursive loop due to the counter:

In [10]:
print(meta.standardised_query(pl, "explain why donald is mortal")[0]['Output'])
print(meta.standardised_query(pl, "explain why peter is a teacher")[0]['Output'])

Sorry, I don't think this is the case
Sorry, I don't think this is the case


### Negation
We can add a new rule to show that donald actually isn't happy, this shows the remove_conflicting_rules predicate working, as the `happy(donald):-true.` rule is removed.

In [11]:
_ = meta.standardised_query(pl, "donald is not happy")[0]['Output']
print(meta.standardised_query(pl, "tell me everything")[0]['Output'])

peter is human. every human is mortal. every teacher is happy. donald is not happy. donald is not happy


I have not yet been able to figure out why it is adding two rules to the prolexa plus stored rules, it only adds one to basic prolexa, there is only one assertz in the basic prolog .pl file? 

The following code cell demonstrates that adding `happy(donald):-true.` removes the `not(happy(donald)):-true.` rules from the stored rules.

In [14]:
_ = meta.standardised_query(pl, "donald is happy")[0]['Output']
print(meta.standardised_query(pl, "tell me everything")[0]['Output'])
_ = meta.standardised_query(pl, "donald is not happy")[0]['Output']

peter is human. every human is mortal. every teacher is happy. donald is happy


Adding that Donald isn't happy shows that this rule works with "Who" and "Explain" via the added prove_rb predicates.

In [12]:
print(meta.standardised_query(pl, "who is not a teacher")[0]['Output'])

b'donald is not a teacher'


In [13]:
print(meta.standardised_query(pl, "explain why donald is not a teacher")[0]['Output'])

donald is not happy; every teacher is happy; therefore donald is not a teacher


### For further testing

In [9]:
input = 'tell me about Peter'  #@param {type:"string"}
print(input)
first_answer = meta.standardised_query(pl, input)[0]['Output']
print(first_answer)

tell me about Peter
peter is human. peter is mortal
